# Getting Started with Valor

## Introduction

Valor is a centralized evaluation store which makes it easy to measure, explore, and rank model performance. Valor empowers data scientists and engineers to evaluate the performance of their machine learning pipelines and use those evaluations to make better modeling decisions in the future. For a conceptual introduction to Valor, [check out our project overview](https://striveworks.github.io/valor/).

In this notebook, we'll introduce Valor's high-level abstractions and walk through a computer vision-oriented example of how you can use Valor to evaluate model performance. For task-specific examples, please see our follow-up notebooks below:
- [Tabular classification](https://github.com/Striveworks/valor/blob/main/examples/classification/tabular.ipynb)
- [Object detection](https://github.com/Striveworks/valor/blob/main/examples/object-detection/coco-yolo.ipynb)
- [Semantic segmentation](https://github.com/Striveworks/valor/blob/main/examples/semantic-segmentation/coco-yolo.ipynb)

Before using this notebook, please ensure that the Valor service is running on your machine (for start-up instructions, [click here](https://striveworks.github.io/valor/getting_started/)). To connect to a non-local instance of Valor, update `client = Client("http://0.0.0.0:8000")` in the first code block to point to the correct URL.

## High-Level Workflow

Valor is equipped to handle a wide variety of supervised learning tasks thanks to its six core abstractions. We can think of these abstractions as being split into two categories:
- **Dataset**: When describing our actual dataset, we define a `Dataset` containing a list of `GroundTruths` which, in turn, are made up of `Datums` and `Annotations`.
- **Model**: When describing our model outputs, we define a `Model` containing a list of `Predictions` which, in turn, are also made up of `Datums` and `Annotations`. We then link our `Model` to a `Dataset` when finalizing the model.

After we define both our dataset inputs and model outputs, Valor will make it easy to calculate and store our evaluation metrics. Let's start by describing our dataset.

## Defining Our Dataset

To begin, we import all needed packages and connect to our Valor API. For instructions on setting up your API, please see [our docs here](https://striveworks.github.io/valor/getting_started/).

In [1]:
from pathlib import Path

from valor import (
    connect,
    Client,
    Dataset,
    Model,
    Datum,
    Annotation,
    GroundTruth, 
    Prediction,
    Label,
)
from valor.schemas import (
    BoundingBox, 
    Polygon,
    Raster,
)
from valor.enums import TaskType

# connect to the Valor API
connect("http://0.0.0.0:8000")
client = Client()

Successfully connected to host at http://0.0.0.0:8000/


Next, we define our `Dataset` in Valor.

In [2]:
dataset = Dataset.create(  
    name="myDataset",
    metadata={        # optional, metadata can take `str`, `int`, `float` value types.
        "some_string": "hello_world",
        "some_number": 1234,
        "a_different_number": 1.234,
    },
)

To describe the various objects in our `Dataset`, we'll associate a list of `GroundTruths` (made up of `Annotations` and `Datums`) to the `Dataset` we defined above. Note that Valor doesn't actually store any images, and that the `Annotations` we use will vary by our task type (i.e., object detection, semantic segmentation, etc.). For demonstrative purposes, we'll create `GroundTruths` for four different learning tasks in this notebook.

### Creating Object Detection GroundTruths

In [3]:
def create_groundtruth_from_object_detection_dict(element: dict):
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=Path(element["path"]).stem,
        metadata={
            "path": element["path"] 
        }
    )

    # create Annotations
    annotations = [
        Annotation(
            task_type=TaskType.OBJECT_DETECTION,
            labels=[Label(key="class_label", value=annotation["class_label"])],
            bounding_box=BoundingBox.from_extrema(
                xmin=annotation["bbox"]["xmin"],
                xmax=annotation["bbox"]["xmax"],
                ymin=annotation["bbox"]["ymin"],
                ymax=annotation["bbox"]["ymax"],
            )
        )
        for annotation in element["annotations"]
        if len(annotation) > 0
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

image_object_detections = [
    {"path": "a/b/c/img3.png", "annotations": [{"class_label": "dog", "bbox": {"xmin": 16, "ymin": 130, "xmax": 70, "ymax": 150}}, {"class_label": "person", "bbox": {"xmin": 89, "ymin": 10, "xmax": 97, "ymax": 110}}]},
    {"path": "a/b/c/img4.png", "annotations": [{"class_label": "cat", "bbox": {"xmin": 500, "ymin": 220, "xmax": 530, "ymax": 260}}]},
    {"path": "a/b/c/img5.png", "annotations": []}
]


for element in image_object_detections:
    # create ground truth
    groundtruth = create_groundtruth_from_object_detection_dict(element)

    # add ground truth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

{'datum': {'uid': 'img3', 'metadata': {'path': {'type': 'string', 'value': 'a/b/c/img3.png'}}}, 'annotations': [{'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': None}], 'metadata': {}, 'bounding_box': [[(16, 130), (70, 130), (70, 150), (16, 150), (16, 130)]], 'polygon': None, 'raster': None, 'embedding': None}, {'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'person', 'score': None}], 'metadata': {}, 'bounding_box': [[(89, 10), (97, 10), (97, 110), (89, 110), (89, 10)]], 'polygon': None, 'raster': None, 'embedding': None}]}
{'datum': {'uid': 'img4', 'metadata': {'path': {'type': 'string', 'value': 'a/b/c/img4.png'}}}, 'annotations': [{'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'cat', 'score': None}], 'metadata': {}, 'bounding_box': [[(500, 220), (530, 220), (530, 260), (500, 260), (500, 220)]], 'polygon': None, 'raster': None, 'embedding': None}]}
{'datum': {'uid': 'img5', 'metadat

/home/czaloom/valor/.env-velour/lib/python3.10/site-packages/valor/coretypes.py:1241: UserWarning: GroundTruth for datum with uid `img5` contains no annotations.
  warnings.warn(


### Creating Image Classification GroundTruths

In [4]:
def create_groundtruth_from_image_classification_dict(element: dict):
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=Path(element["path"]).stem,
        metadata={
            "path": element["path"]
        }
    )

    # create Annotation
    annotations = [
        Annotation(
            task_type=TaskType.CLASSIFICATION,
            labels=[
                Label(key=key, value=value)
                for label in element["annotations"]
                for key, value in label.items()
            ]
        )
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

image_classifications = [
    {"path": "a/b/c/img1.png", "annotations": [{"class_label": "dog"}]},
    {"path": "a/b/c/img2.png", "annotations": [{"class_label": "cat"}]}
]

for element in image_classifications:
    # create ground truth
    groundtruth = create_groundtruth_from_image_classification_dict(element)

    # add ground truth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

{'datum': {'uid': 'img1', 'metadata': {'path': {'type': 'string', 'value': 'a/b/c/img1.png'}}}, 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'raster': None, 'embedding': None}]}
{'datum': {'uid': 'img2', 'metadata': {'path': {'type': 'string', 'value': 'a/b/c/img2.png'}}}, 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'class_label', 'value': 'cat', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'raster': None, 'embedding': None}]}


### Creating Image Segmentation GroundTruths

In [5]:
def create_groundtruth_from_image_segmentation_dict(element: dict):
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=Path(element["path"]).stem,
        metadata={
            "path": element["path"] 
        }
    )

    # create Annotations
    annotations = [
        Annotation(
            task_type=TaskType.SEMANTIC_SEGMENTATION,
            labels=[Label(key="class_label", value=annotation["class_label"])],
            raster=Raster.from_geometry(
                geometry=Polygon(
                    [
                        [
                            (pt['x'], pt['y'])
                            for pt in [*subpolygon, subpolygon[0]]
                        ]
                        for subpolygon in annotation["contour"]
                    ]
                ),
                height=100,
                width=100,
            )
        )
        for annotation in element["annotations"]
        if len(annotation["contour"]) > 0
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

image_segmentations = [
    {"path": "a/b/c/img6.png", "annotations": [{"class_label": "dog", "contour": [[{"x": 10.0, "y": 15.5}, {"x": 20.9, "y": 50.2}, {"x": 25.9, "y": 28.4}]]}]},
    {"path": "a/b/c/img7.png", "annotations": [{"class_label": "cat", "contour": [[{"x": 97.2, "y": 40.2}, {"x": 33.33, "y": 44.3}, {"x": 10.9, "y": 18.7}]]}]},
    {"path": "a/b/c/img8.png", "annotations": [{"class_label": "car", "contour": [[{"x": 10.0, "y": 15.5}, {"x": 20.9, "y": 50.2}, {"x": 25.9, "y": 28.4}], [{"x": 60.0, "y": 15.5}, {"x": 70.9, "y": 50.2}, {"x": 75.9, "y": 28.4}]]}]}
]

for element in image_segmentations:
    # create ground truth
    groundtruth = create_groundtruth_from_image_segmentation_dict(element)

    # add ground truth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

ClientException: {"name": "InternalError", "detail": "(psycopg2.errors.InternalError_) rt_raster_iterator: The set of rasters provided (custom extent included, if appropriate) do not have the same alignment\nCONTEXT:  SQL function \"st_mapalgebra\" statement 1\n\n[SQL: INSERT INTO annotation (datum_id, model_id, task_type, meta, created_at, box, polygon, raster, embedding_id) VALUES (%(datum_id)s, %(model_id)s, %(task_type)s, %(meta)s, now(), ST_GeomFromEWKT(%(box)s), ST_GeomFromEWKT(%(polygon)s), (SELECT ST_MapAlgebra(ST_AddBand(ST_MakeEmptyRaster(%(ST_MakeEmptyRaster_1)s, %(ST_MakeEmptyRaster_2)s, %(ST_MakeEmptyRaster_3)s, %(ST_MakeEmptyRaster_4)s, %(ST_MakeEmptyRaster_5)s, %(ST_MakeEmptyRaster_6)s, %(ST_MakeEmptyRaster_7)s, %(ST_MakeEmptyRaster_8)s, %(ST_MakeEmptyRaster_9)s), %(ST_AddBand_1)s), ST_AsRaster(ST_GeomFromText(%(ST_GeomFromText_1)s), %(ST_AsRaster_1)s, %(ST_AsRaster_2)s, %(ST_AsRaster_3)s, %(ST_AsRaster_4)s, %(ST_AsRaster_5)s), %(ST_MapAlgebra_2)s, %(ST_MapAlgebra_3)s, %

### Creating Text Classification GroundTruths

In [ ]:
def create_groundtruth_from_text_classification_dict(element: dict):
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=Path(element["path"]).stem,
        metadata={
            "path": element["path"],
            "context": element["annotations"][0]["sentiment"]["context"]
        }
    )

    # create Annotation
    annotations = [
        Annotation(
            task_type=TaskType.CLASSIFICATION,
            labels=[
                Label(
                    key="label", 
                    value=element["annotations"][0]["sentiment"]["label"]
                )
            ]
        )
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

text_classifications = [
    {"path": "a/b/c/text1.txt", "annotations": [{"sentiment": {"context": "Is the content of this product review postive?", "label": "positive"}}]}
]

for element in text_classifications:
    # create ground truth
    groundtruth = create_groundtruth_from_text_classification_dict(element)

    # add ground truth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

### Finalizing Our Dataset

Now that we've created all of our `GroundTruth` objects, we finalize our `Dataset` such that it's ready for evaluation. Valor makes finalization a requirement for traceability purposes: we want you to feel confident that a finalized `Dataset` or `Model` won't change over any length of time.



In [ ]:
dataset.finalize()

## Defining Our Model

Now that we've described our dataset, the next step is to define our model and subsequent predictions. Again, for demonstrative purposes, we'll define predictions for four separate task types in this notebook.

In [ ]:
model = Model.create(
    name="myModel",
    metadata={
        "foo": "bar",
        "some_number": 4321,
    },
)

### Creating Object Detection Predictions

In [ ]:
# populate a dictionary mapping Datum UIDs to datums for all of the datums in our dataset
datums_by_uid = {
    datum.uid: datum
    for datum in dataset.get_datums()
}

def create_prediction_from_object_detection_dict(element: dict, datums_by_uid:dict) -> Prediction:
    
    # get datum from dataset using filename
    uid=Path(element["path"]).stem
    datum = datums_by_uid[uid]

    # create Annotations
    annotations = [
        Annotation(
            task_type=TaskType.OBJECT_DETECTION,
            labels=[
                Label(key="class_label", value=label["class_label"], score=label["score"])
                for label in annotation["labels"]
            ],
            bounding_box=BoundingBox.from_extrema(
                xmin=annotation["bbox"]["xmin"],
                xmax=annotation["bbox"]["xmax"],
                ymin=annotation["bbox"]["ymin"],
                ymax=annotation["bbox"]["ymax"],
            )
        )
        for annotation in element["annotations"]
        if len(annotation) > 0
    ]

    # create and return Prediction
    return Prediction(
        datum=datum,
        annotations=annotations,
    )

object_detections = [
    {"path": "a/b/c/img3.png", "annotations": [
        {"labels": [{"class_label": "dog", "score": 0.8}, {"class_label": "cat", "score": 0.1}, {"class_label": "person", "score": 0.1}], "bbox": {"xmin": 16, "ymin": 130, "xmax": 70, "ymax": 150}}, 
        {"labels": [{"class_label": "dog", "score": 0.05}, {"class_label": "cat", "score": 0.05}, {"class_label": "person", "score": 0.9}], "bbox": {"xmin": 89, "ymin": 10, "xmax": 97, "ymax": 110}}
    ]},
    {"path": "a/b/c/img4.png", "annotations": [
        {"labels": [{"class_label": "dog", "score": 0.8}, {"class_label": "cat", "score": 0.1}, {"class_label": "person", "score": 0.1}], "bbox": {"xmin": 500, "ymin": 220, "xmax": 530, "ymax": 260}}
    ]},
    {"path": "a/b/c/img5.png", "annotations": []}
]

for element in object_detections:
    # create prediction
    prediction = create_prediction_from_object_detection_dict(element, datums_by_uid=datums_by_uid)

    # add prediction to model
    model.add_prediction(dataset, prediction)
    
    print(prediction)

### Creating Image Classification Predictions

In [ ]:
def create_prediction_from_image_classification_dict(element: dict, datums_by_uid:dict) -> Prediction:
    
    # get datum from dataset using filename
    uid=Path(element["path"]).stem
    datum = datums_by_uid[uid]

    # create Annotation
    annotations = [
        Annotation(
            task_type=TaskType.CLASSIFICATION,
            labels=[
                Label(key="class_label", value=label["class_label"], score=label["score"])
                for label in element["annotations"]
            ]
        )
    ]

    # create and return Prediction
    return Prediction(
        datum=datum,
        annotations=annotations,
    )

image_classifications = [
    {"path": "a/b/c/img1.png", "annotations": [{"class_label": "dog", "score": 0.9}, {"class_label": "cat", "score": 0.1}]},
    {"path": "a/b/c/img2.png", "annotations": [{"class_label": "dog", "score": 0.1}, {"class_label": "cat", "score": 0.9}]}
]

for element in image_classifications:
    # create prediction
    prediction = create_prediction_from_image_classification_dict(element, datums_by_uid=datums_by_uid)

    # add prediction to dataset
    model.add_prediction(dataset, prediction)
    
    print(prediction)

### Creating Image Segmentation Predictions

In [ ]:
def create_prediction_from_image_segmentation_dict(element: dict, datums_by_uid: dict) -> Prediction:
    
    # get datum from dataset using filename
    uid=Path(element["path"]).stem
    datum = datums_by_uid[uid]


    # create Annotations
    annotations = [
        Annotation(
            task_type=TaskType.SEMANTIC_SEGMENTATION,
            labels=[
                Label(key="class_label", value=annotation["class_label"])
            ],
            raster=Raster.from_geometry(
                geometry=Polygon(
                    [
                        [
                            (pt['x'], pt['y'])
                            for pt in [*subpolygon, subpolygon[0]]
                        ]
                        for subpolygon in annotation["contour"]
                    ]
                ),
                height=100,
                width=100,
            )
        )
        for annotation in element["annotations"]
        if len(annotation["contour"]) > 0
    ]

    # create and return Prediction
    return Prediction(
        datum=datum,
        annotations=annotations,
    )

image_segmentations = [
    {
        "path": "a/b/c/img6.png", 
        "annotations": [
            {
                "class_label": "dog",
                "contour": [[{"x": 10.0, "y": 15.5}, {"x": 20.9, "y": 50.2}, {"x": 25.9, "y": 28.4}]]
            }
        ]
    },
    {
        "path": "a/b/c/img7.png", 
        "annotations": [
            {
                "class_label": "cat",
                "contour": [[{"x": 97.2, "y": 40.2}, {"x": 33.33, "y": 44.3}, {"x": 10.9, "y": 18.7}]]
            }
        ]   
    },
    {
        "path": "a/b/c/img8.png", 
        "annotations": [
            {
                "class_label": "car",
                "contour": [[{"x": 10.0, "y": 15.5}, {"x": 20.9, "y": 50.2}, {"x": 25.9, "y": 28.4}], [{"x": 60.0, "y": 15.5}, {"x": 70.9, "y": 50.2}, {"x": 75.9, "y": 28.4}]]
            }
        ]
    }
]


for element in image_segmentations:
    # create prediction
    prediction = create_prediction_from_image_segmentation_dict(element, datums_by_uid=datums_by_uid)

    # add prediction to model
    model.add_prediction(dataset, prediction)
    
    print(prediction)

### Creating Text Classification Predictions

In [ ]:
def create_prediction_from_text_classification_dict(element: dict, datums_by_uid:dict) -> Prediction:
    
    # get datum from dataset using filename
    uid=Path(element["path"]).stem
    datum = datums_by_uid[uid]

    # create Annotation
    annotations = [
        Annotation(
            task_type=TaskType.CLASSIFICATION,
            labels=[
                Label(
                    key="label", 
                    value=label["label"],
                    score=label["score"],
                )
                for label in element["annotations"][0]["sentiment"]["labels"]
            ]
        )
    ]

    # create and return Prediction
    return Prediction(
        datum=datum,
        annotations=annotations,
    )

text_classifications = [
    {
        "path": "a/b/c/text1.txt",
        "annotations": [
            {"sentiment": 
                {
                    "context": "Is the content of this product review postive?", 
                    "labels": [
                        {"label": "positive", "score": 0.8},
                        {"label": "negative", "score": 0.2}
                    ]
                }
            }
        ]
    }
]


for element in text_classifications:
    # create prediction
    prediction = create_prediction_from_text_classification_dict(element, datums_by_uid=datums_by_uid)

    # add prediction to model
    model.add_prediction(dataset, prediction)
    
    print(prediction)

### Finalizing Our Model

Now that we've created all of our `Prediction` objects, we finalize our `Model` such that it's ready for evaluation. When finalizing our `Model`, we pass in the `Dataset` object that we want to link it to.

In [ ]:
model.finalize_inferences(dataset)

## Exploring Our Objects

Now that we've finalized our `Dataset` and `Model`, we can explore all of the objects stored in Valor before running our evaluations.

### Client Exploration

In [ ]:
client.get_datasets()

In [ ]:
client.get_models()

### Dataset Exploration

In [ ]:
for datum in dataset.get_datums():
    print(datum)

In [ ]:
for datum in dataset.get_datums():
    groundtruth = dataset.get_groundtruth(datum)
    print(groundtruth)

In [ ]:
for label in dataset.get_labels():
    print(label)

### Model Exploration

In [ ]:
for datum in dataset.get_datums():
    print(model.get_prediction(dataset, datum))

In [ ]:
for label in model.get_labels():
    print(label)

## Evaluating Performance

Finally, we'll use our Valor abstractions to evaluate model performance. For more detailed, task-specific examples, see our follow-up notebooks at the links below:
- [Tabular classification](https://github.com/Striveworks/valor/blob/main/examples/classification/tabular.ipynb)
- [Object detection](https://github.com/Striveworks/valor/blob/main/examples/object-detection/coco-yolo.ipynb)
- [Semantic segmentation](https://github.com/Striveworks/valor/blob/main/examples/semantic-segmentation/coco-yolo.ipynb)

### Evaluating Detections

In [ ]:
eval_objdet = model.evaluate_detection(dataset)
eval_objdet.wait_for_completion()
eval_objdet.metrics

### Evaluating Classifications

Note that running the code below evaluates both our text classifications as well as our image classifications. If we only wanted to evaluate one type of classification task, we could use `evaluation_classification`'s `filters` argument to specify which type of labels to evaluate.

In [ ]:
eval_clf = model.evaluate_classification(dataset)
eval_clf.wait_for_completion()
eval_clf.metrics

### Evaluating Segmentations

In [ ]:
eval_semseg = model.evaluate_segmentation(dataset)
eval_semseg.wait_for_completion()
eval_semseg.metrics